In [1]:
#Libraries
#1) Importing Data
import os
from scipy.io import loadmat #Importing Matlab data

#2) Dataset Manipulation
import pandas as pd

#3) Mathematical/Statistical computation
import numpy as np
import numpy.ma as ma #Correlation while ignoring the missing data
import statistics as stat

#4) Plotting
import matplotlib
import matplotlib.pyplot as plt
# -> cf) magic function for IPython and Jupyter Notebook to show the graph well
# -> cf) In original Python, % means remainder from the division so below will not work
# -> cf) %matplotlib inline 
import matplotlib.dates as mdates #Date Library
from matplotlib.patches import Rectangle

#5) Regressions 
import statsmodels.formula.api as sm # Regression with Formula
import statsmodels.api as sm
from linearmodels.iv import IV2SLS
from linearmodels.iv import results
from linearmodels.compat.statsmodels import Summary
from linearmodels.compat.statsmodels import summary
from linearmodels.iv import compare
#from linearmodels.datasets import mroz #Why Do We Need?
#from linearmodels import PanelOLS #Maybe we do not need
import statsmodels.formula.api as smf #For Newey-West Covariance Matrix

#6) Regression Results to Tex Output
from statsmodels.iolib.summary2 import summary_col # Regression Output Table
#import tabulate
#from tabulate import tabulate
#print(tabulate([['Alice', 24], ['Bob', 19]], headers=['Name', 'Age'], tablefmt='orgtbl'))

#7) Math
import math

#8) Random Number Generator
import random

#9) f-SOLVER
import scipy
from scipy import stats
from scipy.stats import lognorm
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint
from scipy import interpolate

#10) numba
from numba import jit, jitclass, njit, prange, int64, float64
import numpy
import math
import time
from scipy.stats import norm
from collections import OrderedDict
import sys

In [2]:
#0. Read parameters for the shocks form parameter file.
os.chdir('C:\\Users\\Owner\\Dropbox\\1. Research\\1. Risks in Household Portfolio Choice through Jeonse System\\Coding')
parameterset=pd.read_csv('parameter.csv')  
FullshockFinal=pd.read_csv('FullShockFinal1.csv')  
lambdaPS=pd.read_csv('survivalprob.csv')
LIncomeGrowthProcess=pd.read_csv('LIncomeGrowthProcess.csv')

In [3]:
#2. Paramters spaces

#2.1. Household Problem Parameters
omega=parameterset['Parameter'][0] #housing versus non-durable consumption
rho=parameterset['Parameter'][1] #inverse of the elasticity of intertemporal substitution 
gamma=parameterset['Parameter'][2] #CRRA risk aversion
beta=parameterset['Parameter'][3] #discount factor
xi=parameterset['Parameter'][4] #probability of moving - Vestman 2019 different for each age
alpha_f=parameterset['Parameter'][5] #bequest strength (annuity factor)
beta_b=parameterset['Parameter'][6] #bequest strength (discount factor)
T_b=parameterset['Parameter'][7] #bequest strength (time period)
lambda_r=parameterset['Parameter'][8] #ratio of retirement income to the last working age income
R_f=parameterset['Parameter'][9] #risk free rate
mu_r=parameterset['Parameter'][10] #risk premium 
mu_h=parameterset['Parameter'][11] #expected growth of house price
tau_r=parameterset['Parameter'][12] #ratio of yearly rent to house price
Jbar=parameterset['Parameter'][13] #ratio of Jeonse deposit to house price -> Refer to the R.file
delta_J=parameterset['Parameter'][14] #downpayment ratio for Jeonse
phi_J=parameterset['Parameter'][15] #Jeonse contract cost 
chi=parameterset['Parameter'][16] #maintenance fee
delta=parameterset['Parameter'][17] #downpayment ratio for home purchase
phi_b=parameterset['Parameter'][18] #House purchase cost
phi=parameterset['Parameter'][19] #selling cost
Hbar=parameterset['Parameter'][20] #minimum house value (lower than it, rent or jeonse)
delta_v=parameterset['Parameter'][21] #standard deviation of idiosyncratic labor income shock
delta_ep=parameterset['Parameter'][22] #standard deviation of idiosyncratic stock market return
delta_n=parameterset['Parameter'][23] #standard deviation of housing return
corrHS=parameterset['Parameter'][24] # correlation between housing return and stock return correlation
eta_en=parameterset['Parameter'][25] # coefficient in front of stock return shock so that it goes into housing return equation for correlation.
corrYS=parameterset['Parameter'][26] # correlation between labor income and stock return
corrHY=parameterset['Parameter'][27] # correlation between housing return and labor income
eta_nn=parameterset['Parameter'][28] # coefficient for shock correlated to housing return and exist in labor income shock part
eta_eo=parameterset['Parameter'][29] # coefficient for shock correlated to stock return and exist in labor income shock part
LT=parameterset['Parameter'][30] #Longevity 20 - 80
b_b=parameterset['Parameter'][31] #bequest constant multiplier

#2.2. Life Cycle Parameters
LIncomeGrowthProcess=np.array(LIncomeGrowthProcess['0']) #deterministic labor income growth profile
lambdaPS=np.array(lambdaPS['0']) #survival probability conditional on age
#lambdaPS=np.array(lambdaPS)

#2.3. Shock Processes (i.i.d) in my case
FullshockFinal=pd.DataFrame(FullshockFinal) #Possible shock realization and probability 
LaborIncomeShock=np.array(FullshockFinal.YgridV+FullshockFinal.SgridV_Y+FullshockFinal.HgridV_Y, dtype=float)
HousePriceShock=np.array(FullshockFinal.HgridV+FullshockFinal.SgridV_H, dtype=float)
StockReturnShock=np.array(FullshockFinal.SgridV, dtype=float)
Probability=np.array(FullshockFinal.Prob, dtype=float)

In [4]:
#3. Setting up the grid for (y,h)

# Grid for y (wage / cash-in-hand)
ny            = int(100);
ymin          = 0.1;
ymax          = 4.0;
# Function to construct the grid for laborincome/cash-in-hand (y)
ygrid = numpy.zeros(ny);
ystep = (ymax - ymin) /(ny - 1);
it = 0;
for i in range(0,ny):
    ygrid[i] = ymin + it*ystep;
    it = it+1;

# Grid for h (House-value / cash-in-hand)
nh            = int(100);
hmin          = 0.5;
hmax          = 20.0;
# Function to construct the grid for housevalue/cash-in-hand (h)
hgrid = np.zeros(nh);
hstep = (hmax - hmin) /(nh - 1);
it = 0;
for i in range(0,nh):
    hgrid[i] = hmin + it*hstep;
    it = it+1;

In [5]:
#4. Value Function and Optimal Policy Function
LT=int(LT)

# Initialize value function V
V_r     = numpy.zeros((LT+1, ny))
V_j     = numpy.zeros((LT+1, ny))
V_p     = numpy.zeros((LT+1, ny))
V_nonow = numpy.zeros((LT+1, ny))
V_s     = numpy.zeros((LT+1, ny, nh))
V_ow    = numpy.zeros((LT+1, ny, nh))

#Optimal Policy for c,h,a,alpha - renter
c_r      = numpy.zeros((LT+1, ny))
h_r      = numpy.zeros((LT+1, ny))
a_r      = numpy.zeros((LT+1, ny))
alpha_r  = numpy.zeros((LT+1, ny))

#Optimal Policy for c,h,a,alpha - jeonse
c_j      = numpy.zeros((LT+1, ny))
h_j      = numpy.zeros((LT+1, ny))
a_j      = numpy.zeros((LT+1, ny))
alpha_j  = numpy.zeros((LT+1, ny))

#Optimal Policy for c,h,a,alpha - purchaser
c_p      = numpy.zeros((LT+1, ny))
h_p      = numpy.zeros((LT+1, ny))
a_p      = numpy.zeros((LT+1, ny))
alpha_p  = numpy.zeros((LT+1, ny))

#Optimal Policy for Housing Tenure - Nonowner
ht_nonow = numpy.zeros((LT, ny))

#Optimal Policy for c,h,a,alpha - stayer
c_s      = numpy.zeros((LT+1, ny, nh))
h_s      = numpy.zeros((LT+1, ny, nh))
a_s      = numpy.zeros((LT+1, ny, nh))
alpha_s  = numpy.zeros((LT+1, ny, nh))

#Optimal Policy for Housing Tenure - Owner
ht_ow    = numpy.zeros((LT, ny, nh))

In [6]:
#4. Comming up with class for state vectors (state vars + parameters + shocks)
specs = OrderedDict()
#the number of states
specs['ind'] = int64 #possible total states cases (y,h)
specs['ny'] = int64 #possible cases for y
specs['nh'] = int64 #possible cases for h
specs['LT'] = int64 #longevity 20+LT - maximum age in model
specs['age'] = int64 #current age
specs['cy'] = float64 #current y state
specs['ch'] = float64 #curren h state
#Shocks structure
specs['Probability'] = float64[:] #probability for each shock case
specs['LaborIncomeShock'] = float64[:] #labor income shock value for each shock case
specs['HousePriceShock'] = float64[:] #house price shock value for each shock case
specs['StockReturnShock'] = float64[:] #stock return shock value for each shock case
specs['ygrid'] = float64[:]
specs['hgrid'] = float64[:]
#household problem parameters
specs['omega']    = float64 #housing versus non-durable consumption    
specs['rho']      = float64 #inverse of the elasticity of intertemporal substitution       
specs['gamma']    = float64 #CRRA risk aversion         
specs['beta']     = float64 #discount factor        
specs['xi']       = float64 #probability of moving - Vestman 2019 different for each age     
specs['alpha_f']  = float64 #bequest strength (annuity factor)         
specs['beta_b']   = float64 #bequest strength (discount factor)          
specs['T_b']      = int64   #bequest strength (time period)        
specs['lambda_r'] = float64 #ratio of retirement income to the last working age income          
specs['R_f']      = float64 #risk free rate      
specs['mu_r']     = float64 #risk premium        
specs['mu_h']     = float64 #expected growth of house price         
specs['tau_r']    = float64 #ratio of yearly rent to house price          
specs['Jbar']     = float64 #ratio of Jeonse deposit to house price -> Refer to the R.file       
specs['delta_J']  = float64 #downpayment ratio for Jeonse           
specs['phi_J']    = float64 #Jeonse contract cost         
specs['chi']      = float64 #maintenance fee         
specs['delta']    = float64 #downpayment ratio for home purchase              
specs['phi_b']    = float64 #House purchase cost
specs['phi']      = float64 #House selling cost
specs['Hbar']     = int64 #minimum house value (lower than it, rent)
specs['b_b']      = float64  #bequest constant multiplier
#Life-cycle-profile
specs['LIncomeGrowthProcess'] = float64[:]
specs['lambdaPS'] = float64[:]
#valuefunctions
specs['V_r'] = float64[:,:] #age, labor income rateio
specs['V_j'] = float64[:,:] #age, labor income rateio
specs['V_p'] = float64[:,:] #age, labor income rateio
specs['V_nonow'] = float64[:,:] #age, labor income rateio
specs['V_s'] = float64[:,:,:] #age, labor income rateio, house value ratio
specs['V_ow'] = float64[:,:,:]  #age, labor income rateio, house value ratio

In [7]:
# Object(class) structure for state and exogenous variables
@jitclass(specs)
class modelState(object):
    def __init__(self,ind,ny,nh,LT,age,cy,ch,
                 Probability,LaborIncomeShock,HousePriceShock,StockReturnShock,ygrid,hgrid,
                 omega,rho,gamma,beta,xi,alpha_f,beta_b,T_b,lambda_r,R_f,mu_r,mu_h,tau_r,Jbar,delta_J,phi_J,chi,delta,phi_b,phi,Hbar,b_b,
                 LIncomeGrowthProcess,lambdaPS,
                 V_r,V_j,V_p,V_nonow,V_s,V_ow
                ):
        self.ind                   = ind                 
        self.ny                    = ny                  
        self.nh                    = nh                    
        self.LT                    = LT                       
        self.age                   = age
        self.cy                    = cy  
        self.ch                    = ch
        self.Probability           = Probability           
        self.LaborIncomeShock      = LaborIncomeShock       
        self.HousePriceShock       = HousePriceShock         
        self.StockReturnShock      = StockReturnShock         
        self.ygrid                 = ygrid                   
        self.hgrid                 = hgrid                         
        self.omega                 = omega                 
        self.rho                   = rho                   
        self.gamma                 = gamma                   
        self.beta                  = beta                    
        self.xi                    = xi                     
        self.alpha_f               = alpha_f                
        self.beta_b                = beta_b               
        self.T_b                   = T_b                   
        self.lambda_r              = lambda_r                  
        self.R_f                   = R_f                   
        self.mu_r                  = mu_r                     
        self.mu_h                  = mu_h                       
        self.tau_r                 = tau_r                     
        self.Jbar                  = Jbar                          
        self.delta_J               = delta_J                    
        self.phi_J                 = phi_J                  
        self.chi                   = chi                       
        self.delta                 = delta                     
        self.phi_b                 = phi_b                       
        self.phi                   = phi                         
        self.Hbar                  = Hbar                        
        self.b_b                   = b_b                         
        self.LIncomeGrowthProcess  = LIncomeGrowthProcess              
        self.lambdaPS              = lambdaPS                        
        self.V_r                   = V_r                          
        self.V_j                   = V_j                            
        self.V_p                   = V_p                           
        self.V_nonow               = V_nonow                     
        self.V_s                   = V_s                          
        self.V_ow                  = V_ow                                       
        
#here by using self, we define the attribute and method (Calling the sub component)

In [8]:
#5. Solving Value Function - Component-wise

#5.0. Algorithm Structure

#cf) Last period
#All value functions have age dimension which is one-element larger than other objects (policy function, ...)
#This last period contains all zero, which means after the household dies, there is no utility
#Given this last period non-owner and owner value functions, households at their last age solve the problem.

#for every age starting backward from LT-1 (100 age values in array sense)
#   for every state y, 
#     1. solve V_r and policy functions
#     2. solve V_j and policy functions
#     3. solve V_p and policy functions
#     4. solve V_nonow and policy functions
#     for every state h, 
#        5. solve V_s and policy functions
#        6. solve V_ow and policy functions
#Then go back to the 1 again with one less age

In [ ]:
############################################################################################################
############################### Renter's Problem ###########################################################
############################################################################################################

In [45]:
# Rent - Whole Expectation

def value_rent(array, states=state):
    c=array[0]
    h=array[1]
    a=array[2]
    alpha=array[3]
    #0. Calling state vars
    y=states.cy #initial states
    #1. Calling parameters
    age=states.age
    # - Household-Parameters
    omega=states.omega
    gamma=states.gamma
    b_b=states.b_b
    alpha_f=states.alpha_f
    lambda_r=states.lambda_r
    # - Stock Return, Housing Return
    mu_h=states.mu_h
    R_f=states.R_f
    mu_r=states.mu_r
    # - Labor Income
    LIncomeGrowthProcess=states.LIncomeGrowthProcess
    # - Exoenous Shocks
    Probability=states.Probability
    HousePriceShock=states.HousePriceShock
    LaborIncomeShock=states.LaborIncomeShock
    StockReturnShock=states.StockReturnShock
    # - Surviving probability
    lambdaPS=states.lambdaPS
    # - Value Function
    V_nonow=states.V_nonow
    
    #2. Calculating value 
    
    # - Calculate the law of motion as a fct of choice and future shocks - array with shock cases 
    #Housing Return
    NPHPG=np.exp(mu_h+HousePriceShock) 
    #Next Period Gamma
    #Next Period y - depending on your age (retirement)
    if age<45:
        NPGM=a*R_f+alpha*a*(np.exp(np.log(R_f)+mu_r+StockReturnShock)-R_f)+y*np.exp(LIncomeGrowthProcess[age]+LaborIncomeShock)
        NPY=(y*np.exp(LIncomeGrowthProcess[age]+LaborIncomeShock))/NPGM     
    elif age==45:
        NPGM=a*R_f+alpha*a*(np.exp(np.log(R_f)+mu_r+StockReturnShock)-R_f)+lambda_r*y*np.exp(LIncomeGrowthProcess[age]) #At 65 age, next period, your wage does not experience any shock or growth
        NPY=lambda_r*y/NPGM
    else:
        NPGM=a*R_f+alpha*a*(np.exp(np.log(R_f)+mu_r+StockReturnShock)-R_f)+y*np.exp(LIncomeGrowthProcess[age])
        NPY=y/NPGM
    
    # - Expectation
    expectation1=0
    expectation2=0
    for i in range(0,len(Probability)):
        expectation1=expectation1+Probability[i]*((NPGM[i])/(NPHPG[i]**omega))**(1-gamma)
        if age<79:
            expectation2=expectation2+Probability[i]*((NPGM[i]/(NPHPG[i]**omega))*np.interp(NPY[i], ygrid, V_nonow[age+1]))**(1-gamma)
    
    # - Final Value
    if age<79: #Before the last period
        finalvalue=-((((c**(1-omega))*(h**omega))**(1-rho))+beta*(lambdaPS[age])*((expectation2**(1/(1-gamma)))**(1-rho))+beta*(1-lambdaPS[age])*(b_b*(alpha_f*expectation1**(1/(1-gamma)))**(1-rho)))**(1/(1-rho))
    else: #At the last period, there is no next period value function
        finalvalue=-((((c**(1-omega))*(h**omega))**(1-rho))+beta*(1-lambdaPS[age])*(b_b*(alpha_f*expectation1**(1/(1-gamma)))**(1-rho)))**(1/(1-rho))
    
    return finalvalue #to use minimizer later, we put (-) here

In [46]:
x0=np.array([0.4, 1, 0.55, 0.3])
start = time.time()
p=-value_rent(x0)
finish = time.time() - start
print(p)
print(finish)

0.5632644767109687
0.008012533187866211


C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:57: RuntimeWarning: divide by zero encountered in double_scalars


In [47]:
Calculating optimal value
# order of choice variables - c h a alpha
bounds = Bounds([0, 0, 0, 0], [1, np.inf, 1, 1]) #consumption cannot be negative, portfolio cannot be larger than 1
linear_constraint = LinearConstraint([1, tau_r, 1, 0], [-np.inf], [1]) #budget constraint
start = time.time()
#initial value 
x0=np.array([0.4, 1, 0.55, 0.3])
res = minimize(value_rent, x0, method='trust-constr',
              constraints=[linear_constraint], bounds=bounds)
finish = time.time() - start
print(finish)
print(res)
print(res.x)

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:57: RuntimeWarning: divide by zero encountered in double_scalars


0.7326376438140869
 barrier_parameter: 2.560000000000001e-07
 barrier_tolerance: 2.560000000000001e-07
          cg_niter: 36
      cg_stop_cond: 4
            constr: [array([0.99999981]), array([7.49999428e-01, 5.00000311e+00, 2.24581034e-07, 5.42079102e-01])]
       constr_nfev: [0, 0]
       constr_nhev: [0, 0]
       constr_njev: [0, 0]
    constr_penalty: 1.0
  constr_violation: 0.0
    execution_time: 0.6964883804321289
               fun: -1.2372537480497383
              grad: array([-1.21575323, -0.06078757, -0.11566333,  0.        ])
               jac: [array([[1.  , 0.05, 1.  , 0.  ]]), array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])]
   lagrangian_grad: array([-7.00148773e-11,  1.40041506e-09, -2.22044605e-15,  9.46278170e-09])
           message: '`gtol` termination condition is satisfied.'
            method: 'tr_interior_point'
              nfev: 100
              nhev: 0
               nit: 28
             niter: 2

In [48]:
#Saving optimal policy function to array
c_r[age,1]=res.x[0]
h_r[age,1]=res.x[1]
a_r[age,1]=res.x[2]
alpha_r[age,1]=res.x[3]
V_r[age,1]=-res.fun

In [ ]:
############################################################################################################
############################### Jeonse Renter's Problem ####################################################
############################################################################################################

In [51]:
# Jeonse - Whole Expectation
#@njit - how can I use numba?
def value_jeonse(array, states=state):
    c=array[0]
    h=array[1]
    a=array[2]
    alpha=array[3]
    #0. Calling state vars
    y=states.cy #initial states
    #1. Calling parameters
    age=states.age
    # - Household-Parameters
    omega=states.omega
    gamma=states.gamma
    b_b=states.b_b
    alpha_f=states.alpha_f
    Jbar=states.Jbar
    delta_J=states.delta_J
    phi_J=states.phi_J
    lambda_r=states.lambda_r
    # - Stock Return, Housing Return
    mu_h=states.mu_h
    R_f=states.R_f
    mu_r=states.mu_r
    # - Labor Income
    LIncomeGrowthProcess=states.LIncomeGrowthProcess
    # - Exoenous Shocks
    Probability=states.Probability
    HousePriceShock=states.HousePriceShock
    LaborIncomeShock=states.LaborIncomeShock
    StockReturnShock=states.StockReturnShock
    # - Surviving probability
    lambdaPS=states.lambdaPS
    # - Value Function
    V_nonow=states.V_nonow
    
    #2. Calculating value 
    
    # - Calculate the law of motion as a fct of choice and future shocks - array with shock cases 
    #Housing Return
    NPHPG=np.exp(mu_h+HousePriceShock) 
    #Next Period Gamma
    #Next Period y - depending on your age (retirement)
    if age<45:
        NPGM=a*R_f+alpha*a*(np.exp(np.log(R_f)+mu_r+StockReturnShock)-R_f)+y*np.exp(LIncomeGrowthProcess[age]+LaborIncomeShock)+h*Jbar*(1-(1-delta_J)*R_f)
        NPY=(y*np.exp(LIncomeGrowthProcess[age]+LaborIncomeShock))/NPGM     
    elif age==45:
        NPGM=a*R_f+alpha*a*(np.exp(np.log(R_f)+mu_r+StockReturnShock)-R_f)+lambda_r*y*np.exp(LIncomeGrowthProcess[age])+h*Jbar*(1-(1-delta_J)*R_f) #At 65 age, next period, your wage does not experience any shock or growth
        NPY=lambda_r*y/NPGM
    else:
        NPGM=a*R_f+alpha*a*(np.exp(np.log(R_f)+mu_r+StockReturnShock)-R_f)+y*np.exp(LIncomeGrowthProcess[age])+h*Jbar*(1-(1-delta_J)*R_f)
        NPY=y/NPGM
    
    # - Expectation
    expectation1=0
    expectation2=0
    for i in range(0,len(Probability)):
        expectation1=expectation1+Probability[i]*((NPGM[i])/(NPHPG[i]**omega))**(1-gamma)
        if age<79:
            expectation2=expectation2+Probability[i]*((NPGM[i]/(NPHPG[i]**omega))*np.interp(NPY[i], ygrid, V_nonow[age+1]))**(1-gamma)
    
    # - Final Value
    if age<79: #Before the last period
        finalvalue=-((((c**(1-omega))*(h**omega))**(1-rho))+beta*(lambdaPS[age])*((expectation2**(1/(1-gamma)))**(1-rho))+beta*(1-lambdaPS[age])*(b_b*(alpha_f*expectation1**(1/(1-gamma)))**(1-rho)))**(1/(1-rho))
    else: #At the last period, there is no next period value function
        finalvalue=-((((c**(1-omega))*(h**omega))**(1-rho))+beta*(1-lambdaPS[age])*(b_b*(alpha_f*expectation1**(1/(1-gamma)))**(1-rho)))**(1/(1-rho))
    
    return finalvalue #to use minimizer later, we put (-) here

#per calculation time -
#x0=np.array([0.4, 1, 0.55, 0.3])
#start = time.time()
#res=value_jeonse(x0)
#finish = time.time() - start
#print(finish)
#print(res)

In [52]:
#per calculation time -
x0=np.array([0.4, 1, 0.55, 0.3])
start = time.time()
res=value_jeonse(x0)
finish = time.time() - start
print(finish)
print(res)

0.010651350021362305
-0.5734052482693921


C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: RuntimeWarning: divide by zero encountered in double_scalars


In [53]:
# Jeonse-Calculating optimal value

# order of choice variables - c h a alpha
bounds = Bounds([0, 0, 0, 0], [1, np.inf, 1, 1]) #consumption cannot be negative, portfolio cannot be larger than 1
linear_constraint = LinearConstraint([1, (delta_J+phi_J)*Jbar, 1, 0], [-np.inf], [1]) #budget constraint
start = time.time()
#initial value 
x0=np.array([0.4, 1, 0.55, 0.3])
res = minimize(value_jeonse, x0, method='trust-constr',
              constraints=[linear_constraint], bounds=bounds)
finish = time.time() - start
print(finish)
print(res)
print(res.x)

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: RuntimeWarning: divide by zero encountered in double_scalars


0.656566858291626
 barrier_parameter: 5.120000000000003e-08
 barrier_tolerance: 5.120000000000003e-08
          cg_niter: 39
      cg_stop_cond: 4
            constr: [array([0.99999993]), array([7.35168915e-01, 1.13989136e+00, 5.55312713e-08, 4.42957007e-01])]
       constr_nfev: [0, 0]
       constr_nhev: [0, 0]
       constr_njev: [0, 0]
    constr_penalty: 1.0
  constr_violation: 0.0
    execution_time: 0.6051352024078369
               fun: -0.8674617165028768
              grad: array([-0.85263216, -0.19809193, -0.07923236,  0.        ])
               jac: [array([[1.     , 0.23233, 1.     , 0.     ]]), array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])]
   lagrangian_grad: array([-2.22708581e-09,  9.58587407e-09, -2.22044605e-16, -2.54021088e-09])
           message: '`gtol` termination condition is satisfied.'
            method: 'tr_interior_point'
              nfev: 85
              nhev: 0
               nit: 26
           

In [54]:
#Saving optimal policy function to array
c_j[age,1]=res.x[0]
h_j[age,1]=res.x[1]
a_j[age,1]=res.x[2]
alpha_j[age,1]=res.x[3]
V_j[age,1]=-res.fun

In [55]:
alpha_j[age,1]

0.44295700686716855

In [ ]:
####################################################################################################
############################### Buyer's Problem ####################################################
####################################################################################################

In [96]:
# Purchase - Whole Expectation
#@njit
def value_purchase(array, states=state):
    c=array[0]
    h=array[1]
    a=array[2]
    alpha=array[3]
    #0. Calling state vars
    y=states.cy #initial states
    ygrid=states.ygrid
    hgrid=states.hgrid
    #1. Calling parameters
    age=states.age
    # - Household-Parameters
    omega=states.omega
    gamma=states.gamma
    b_b=states.b_b
    alpha_f=states.alpha_f
    Jbar=states.Jbar
    delta_J=states.delta_J
    phi_J=states.phi_J
    phi=states.phi
    delta=states.delta
    xi=states.xi
    lambda_r=states.lambda_r
    # - Stock Return, Housing Return
    mu_h=states.mu_h
    R_f=states.R_f
    mu_r=states.mu_r
    # - Labor Income
    LIncomeGrowthProcess=states.LIncomeGrowthProcess
    # - Exoenous Shocks
    Probability=states.Probability
    HousePriceShock=states.HousePriceShock
    LaborIncomeShock=states.LaborIncomeShock
    StockReturnShock=states.StockReturnShock
    # - Surviving probability
    lambdaPS=states.lambdaPS
    # - Value Function
    V_nonow=states.V_nonow
    V_ow=states.V_ow
    
    #2. Calculating value 
    
    # - Calculate the law of motion as a fct of choice and future shocks - array with shock cases 
    #Housing Return
    NPHPG=np.exp(mu_h+HousePriceShock) 
    
    if age<45:
        #Next Period Gamma
        NPGM=a*R_f+alpha*a*(np.exp(np.log(R_f)+mu_r+StockReturnShock)-R_f)+y*np.exp(LIncomeGrowthProcess[age]+LaborIncomeShock)+h*(NPHPG*(1-phi)-(1-delta)*R_f)
        #Next Period h_t+1,t
        hP=h*(NPHPG/NPGM)
        #Next Period y - depending on your age (retirement)
        NPY=(y*np.exp(LIncomeGrowthProcess[age]+LaborIncomeShock))/NPGM     
    elif age==45:
        NPGM=a*R_f+alpha*a*(np.exp(np.log(R_f)+mu_r+StockReturnShock)-R_f)+lambda_r*y*np.exp(LIncomeGrowthProcess[age])+h*(NPHPG*(1-phi)-(1-delta)*R_f) #At 65 age, next period, your wage does not experience any shock or growth
        hP=h*(NPHPG/NPGM)
        NPY=lambda_r*y/NPGM
    else:
        NPGM=a*R_f+alpha*a*(np.exp(np.log(R_f)+mu_r+StockReturnShock)-R_f)+y*np.exp(LIncomeGrowthProcess[age])+h*(NPHPG*(1-phi)-(1-delta)*R_f)
        hP=h*(NPHPG/NPGM)
        NPY=y/NPGM
    
    # - Expectation
    expectation1=0 # bequest - no change
    expectation2=0 # next value function - moving shock
    expectation3=0 # next value function - no moving shock
    
    #Interpolate the future owner's value function https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp2d.html
    yy, hh =np.meshgrid(ygrid, hgrid)
    V_ow2 = interpolate.interp2d(yy, hh, V_ow[age+1], kind='cubic', bounds_error=True) # - 2d interpolation, when # of grid increases, it become unavailable..
    for i in range(0,len(Probability)):
        expectation1=expectation1+Probability[i]*((NPGM[i])/(NPHPG[i]**omega))**(1-gamma)
        if age<79:
            expectation2=expectation2+Probability[i]*((NPGM[i]/(NPHPG[i]**omega))*np.interp(NPY[i], ygrid, V_nonow[age+1]))**(1-gamma)
            expectation3=expectation3+Probability[i]*((NPGM[i]/(NPHPG[i]**omega))*V_ow2(NPY[i],hP[i]))**(1-gamma)
    
    # - Final Value
    if age<79: #Before the last period
        finalvalue=-((((c**(1-omega))*(h**omega))**(1-rho))
                     +beta*(lambdaPS[age])*xi*((expectation2**(1/(1-gamma)))**(1-rho))
                     +beta*(lambdaPS[age])*(1-xi)*((expectation3**(1/(1-gamma)))**(1-rho))
                     +beta*(1-lambdaPS[age])*(b_b*(alpha_f*expectation1**(1/(1-gamma)))**(1-rho)))**(1/(1-rho))
    else: #At the last period, there is no next period value function
        finalvalue=-((((c**(1-omega))*(h**omega))**(1-rho))+beta*(1-lambdaPS[age])*(b_b*(alpha_f*expectation1**(1/(1-gamma)))**(1-rho)))**(1/(1-rho))
    
    return finalvalue #to use minimizer later, we put (-) here

#per calculation time - 0.004986286163330078
#start = time.time()
#value_jeonse(0.7, 0.3, 0.2, 1)
#finish = time.time() - start
#print(finish)

In [97]:
#per calculation time -
x0=np.array([0.4, 1, 0.55, 0.3])
start = time.time()
res=value_purchase(x0)
finish = time.time() - start
print(finish)
print(res)

0.01786971092224121
-2.636860303264887


In [98]:
# Purchase-Calculating optimal value

# order of choice variables - c h a alpha
bounds = Bounds([0, 0, 0, 0], [1, np.inf, 1, 1]) #consumption cannot be negative, portfolio cannot be larger than 1
linear_constraint = LinearConstraint([1, (chi+delta+phi_b), 1, 0], [-np.inf], [1]) #budget constraint
start = time.time()
#initial value 
x0=np.array([0.4, 1, 0.55, 0.3])
res = minimize(value_purchase, x0, method='trust-constr',
              constraints=[linear_constraint], bounds=bounds)
finish = time.time() - start
print(finish)
print(res)
print(res.x)

1.2189688682556152
 barrier_parameter: 1.0240000000000006e-08
 barrier_tolerance: 1.0240000000000006e-08
          cg_niter: 41
      cg_stop_cond: 0
            constr: [array([0.99999997]), array([0.18233992, 0.32156125, 0.67520841, 0.39336349])]
       constr_nfev: [0, 0]
       constr_nhev: [0, 0]
       constr_njev: [0, 0]
    constr_penalty: 1.0
  constr_violation: 0.0
    execution_time: 1.143982172012329
               fun: -2.0195235565192773
              grad: array([-1.83758438e+00, -8.14049840e-01, -1.83758456e+00, -5.96046448e-08])
               jac: [array([[1.   , 0.443, 1.   , 0.   ]]), array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])]
   lagrangian_grad: array([ 1.98754549e-09,  5.14558378e-09, -4.26703913e-09, -6.75408377e-09])
           message: '`gtol` termination condition is satisfied.'
            method: 'tr_interior_point'
              nfev: 80
              nhev: 0
               nit: 26
             nite

In [59]:
#Saving optimal policy function to array
c_p[age,1]=res.x[0]
h_p[age,1]=res.x[1]
a_p[age,1]=res.x[2]
alpha_p[age,1]=res.x[3]
V_p[age,1]=-res.fun

In [ ]:
####################################################################################################
############################### Stayer's Problem ###################################################
####################################################################################################

In [10]:
# Stayer - Whole Expectation
#@njit
def value_stayer(array, states=state): #be careful, we do not need h here!
    c=array[0]
    a=array[1]
    alpha=array[2]
    #0. Calling state vars
    y=states.cy #initial states
    h=states.ch
    ygrid=states.ygrid
    hgrid=states.hgrid
    #1. Calling parameters
    age=states.age
    # - Household-Parameters
    omega=states.omega
    gamma=states.gamma
    b_b=states.b_b
    alpha_f=states.alpha_f
    Jbar=states.Jbar
    delta_J=states.delta_J
    phi_J=states.phi_J
    phi=states.phi
    delta=states.delta
    xi=states.xi
    lambda_r=states.lambda_r
    # - Stock Return, Housing Return
    mu_h=states.mu_h
    R_f=states.R_f
    mu_r=states.mu_r
    # - Labor Income
    LIncomeGrowthProcess=states.LIncomeGrowthProcess
    # - Exoenous Shocks
    Probability=states.Probability
    HousePriceShock=states.HousePriceShock
    LaborIncomeShock=states.LaborIncomeShock
    StockReturnShock=states.StockReturnShock
    # - Surviving probability
    lambdaPS=states.lambdaPS
    # - Value Function
    V_nonow=states.V_nonow
    V_ow=states.V_ow
    
    #2. Calculating value 
    
    # - Calculate the law of motion as a fct of choice and future shocks - array with shock cases 
    #Housing Return
    NPHPG=np.exp(mu_h+HousePriceShock) 
    
    if age<45:
        #Next Period Gamma
        NPGM=a*R_f+alpha*a*(np.exp(np.log(R_f)+mu_r+StockReturnShock)-R_f)+y*np.exp(LIncomeGrowthProcess[age]+LaborIncomeShock)+h*(NPHPG*(1-phi)-(1-delta)*R_f)
        #Next Period h_t+1,t
        hP=h*(NPHPG/NPGM)
        #Next Period y - depending on your age (retirement)
        NPY=(y*np.exp(LIncomeGrowthProcess[age]+LaborIncomeShock))/NPGM     
    elif age==45:
        NPGM=a*R_f+alpha*a*(np.exp(np.log(R_f)+mu_r+StockReturnShock)-R_f)+lambda_r*y*np.exp(LIncomeGrowthProcess[age])+h*(NPHPG*(1-phi)-(1-delta)*R_f) #At 65 age, next period, your wage does not experience any shock or growth
        hP=h*(NPHPG/NPGM)
        NPY=lambda_r*y/NPGM
    else:
        NPGM=a*R_f+alpha*a*(np.exp(np.log(R_f)+mu_r+StockReturnShock)-R_f)+y*np.exp(LIncomeGrowthProcess[age])+h*(NPHPG*(1-phi)-(1-delta)*R_f)
        hP=h*(NPHPG/NPGM)
        NPY=y/NPGM
    
    # - Expectation
    expectation1=0 # bequest - no change
    expectation2=0 # next value function - moving shock
    expectation3=0 # next value function - no moving shock
    
    #Interpolate the future owner's value function
    yy, hh =np.meshgrid(ygrid, hgrid)
    V_ow2 = interpolate.interp2d(yy, hh, V_ow[age+1], kind='cubic', bounds_error=False) # - 2d interpolation, when # of grid increases, it become unavailable..
    for i in range(0,len(Probability)):
        expectation1=expectation1+Probability[i]*((NPGM[i])/(NPHPG[i]**omega))**(1-gamma)
        if age<79:
            expectation2=expectation2+Probability[i]*((NPGM[i]/(NPHPG[i]**omega))*np.interp(NPY[i], ygrid, V_nonow[age+1]))**(1-gamma)
            expectation3=expectation3+Probability[i]*((NPGM[i]/(NPHPG[i]**omega))*V_ow2(NPY[i],hP[i]))**(1-gamma)

    # - Final Value
    if age<79: #Before the last period
        finalvalue=-((((c**(1-omega))*(h**omega))**(1-rho))
                     +beta*(lambdaPS[age])*xi*((expectation2**(1/(1-gamma)))**(1-rho))
                     +beta*(lambdaPS[age])*(1-xi)*((expectation3**(1/(1-gamma)))**(1-rho))
                     +beta*(1-lambdaPS[age])*(b_b*(alpha_f*expectation1**(1/(1-gamma)))**(1-rho)))**(1/(1-rho))
    else: #At the last period, there is no next period value function
        finalvalue=-((((c**(1-omega))*(h**omega))**(1-rho))+beta*(1-lambdaPS[age])*(b_b*(alpha_f*expectation1**(1/(1-gamma)))**(1-rho)))**(1/(1-rho))
    
    return finalvalue #to use minimizer later, we put (-) here

#per calculation time - 0.004986286163330078
#start = time.time()
#value_jeonse(0.7, 0.3, 0.2, 1)
#finish = time.time() - start
#print(finish)

In [11]:
#per calculation time -
x0=np.array([12412512312412, 0.2, 0.5])
start = time.time()
res=value_stayer(x0)
finish = time.time() - start
print(finish)
print(res)

0.020642757415771484
-0.48255238280309487


In [28]:
# Staying-Calculating optimal value (SLSQP)

# order of choice variables - c h a alpha
bounds = Bounds([0, 0, 0], [1, 1, 1]) #consumption cannot be negative, portfolio cannot be larger than 1
linear_constraint = LinearConstraint([1, 1, 0], [1-(chi+delta-phi)*ch], [1-(chi+delta-phi)*ch]) #budget constraint - given ch!
start = time.time()
#initial value 
x0=np.array([(1-(chi+delta-phi)*ch)/2, (1-(chi+delta-phi)*ch)/2, 0.5]) #initial value contains three arguments!
res = minimize(value_stayer, x0, method='SLSQP', constraints=[linear_constraint],
              bounds=bounds)
finish = time.time() - start
print(finish)
print(res)
print(res.x)

0.3578352928161621
     fun: -1.1042108516313052
     jac: array([ 0.00000000e+00, -1.61664990e+00, -1.58548355e-05])
 message: 'Optimization terminated successfully.'
    nfev: 25
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([7.63282796e-15, 5.84015152e-01, 5.80344522e-01])
[7.63282796e-15 5.84015152e-01 5.80344522e-01]


In [ ]:
# Staying-Calculating optimal value (trust-constr)

# order of choice variables - c a alpha
bounds = Bounds([0, 0, 0], [1, 1, 1]) #consumption cannot be negative, portfolio cannot be larger than 1
linear_constraint = LinearConstraint([1, 1, 0], [1-(chi+delta-phi)*ch], [1-(chi+delta-phi)*ch]) #budget constraint - given ch!
start = time.time()
#initial value 
x0=np.array([(1-(chi+delta-phi)*ch)/2, (1-(chi+delta-phi)*ch)/2, 0.01]) #initial value contains three arguments!
res = minimize(value_stayer, x0, method='trust-constr',
              constraints=[linear_constraint], bounds=bounds)
finish = time.time() - start
print(finish)
print(res)
print(res.x)

In [63]:
#Saving optimal policy function to array
c_s[age,1,1]=res.x[0]
a_s[age,1,1]=res.x[1]
alpha_s[age,1,1]=res.x[2]
V_s[age,1,1]=-res.fun

In [64]:
#Non-Owner's problem
V_nonow[age,1]=max([V_r[age,1],V_j[age,1],V_p[age,1]])
ht_nonow[age,1]=np.argmax([V_r[age,1],V_j[age,1],V_p[age,1]])

In [65]:
#Owner's problem
V_ow[age,1,1]=max([V_nonow[age,1],V_s[age,1,1]])
ht_ow[age,1,1]=np.argmax([V_nonow[age,1],V_s[age,1,1]])